In [37]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sklearn'

## Load data and build test/train split

In [71]:
data = pd.read_csv("./data/RRCA_baseflow.csv")

In [72]:
def test_train_split():
    data = data.sample(frac=1)
    split = int(0.8 * len(data))

    train = data.iloc[:split, :]
    test = data.iloc[split:, :]

    train.to_csv("./data/train.csv", index=None)
    test.to_csv("./data/test.csv", index=None)

# test_train_split()

In [73]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [74]:
split = int(0.8 * len(train))
train = train.sample(frac=1)

validation = train.iloc[split:, :]
train = train.iloc[:split, :]

## Explore data

In [75]:
print(data.columns.values)
print()

for header in data.columns:
    display(header)
    print(f"std: {data[[header]].values.flatten().std()}")
    print(f"avg: {data[[header]].values.flatten().mean()}")
    print(f"min: {data[[header]].values.flatten().min()}")
    print(f"max: {data[[header]].values.flatten().max()}")
    print()

['Date' 'Segment_id' 'x' 'y' 'Evapotranspiration' 'Precipitation'
 'Irrigation_pumping' 'Observed']



'Date'

std: 6287.137751579983
avg: 719206.1753575781
min: 708479
max: 730729



'Segment_id'

std: 52.78791190766203
avg: 126.58989160413059
min: 40
max: 256



...

std: 10.512571361268662
avg: 14.915468539542044
min: 0.0
max: 40.38



'Irrigation_pumping'

std: 0.27371714995088736
avg: -0.090712565133731
min: -3.1647
max: 0.0



'Observed'

std: 56.273487440683574
avg: 23.986753837712268
min: -22.0
max: 747.80328



In [76]:
# Split into Test, Train / Validation
# 1) Split data into test and train files
# 2) While training and evaluating, split train into train and validation